merge long and wide format of unemployment rate data to time-series data

In [1]:
import pandas as pd
import datetime
import re

In [2]:
df_wide = pd.read_csv('alq_wide.csv', index_col=0)
df_long = pd.read_csv('alq_long.csv', index_col=0)

In [3]:
df_wide.shape

(401, 169)

In [4]:
df_long.shape

(67769, 3)

In [5]:
df_long.dtypes

ags5            int64
time_stamp     object
value         float64
dtype: object

## merge

In [6]:
df_final_wide = pd.read_csv('../../final_dfs/for_modeling/df_final_date_wide_2007.csv', index_col=0)
df_final_long = pd.read_csv('../../final_dfs/for_modeling/df_final_date_long_2007.csv', index_col=0)

In [7]:
df_final_wide.shape

(68972, 40)

In [8]:
df_final_long.shape

(543439, 5)

In [9]:
# fix variable name
df_final_wide.rename(
    columns={"employees_social_security_at_residenceemployees_social_security_at_work": 
             "employees_social_security_at_work"}, inplace=True)

In [10]:
def fix_variable(var):
    if var=='employees_social_security_at_residenceemployees_social_security_at_work':
        var = 'employees_social_security_at_work'
        return var
    else:
        return var

In [11]:
df_final_long['variable'] = df_final_long['variable'].apply(fix_variable)

In [12]:
# fix date to first day of the month to merge
def fix_day(date):
    return date.replace(day=1)

### wide

In [13]:
df_final_wide.shape

(68972, 40)

In [14]:
df_wide.shape

(401, 169)

In [15]:
#df_final_wide

In [16]:
df_long['datetime'] = pd.to_datetime(df_long['time_stamp'], format='%Y-%m-%d')

In [17]:
# filter these out to avoid duplicate when merging
#df_long = df_long[df_long['datetime']<datetime.datetime(2019,1,1)]

In [18]:
df_long['date'] = df_long['datetime'].apply(fix_day)

In [19]:
df_long = df_long[['ags5', 'date', 'value']]

In [20]:
df_long.columns = ['ags5', 'date', 'unemployment_rate']

In [21]:
df_long

,ags5,date,unemployment_rate
0,1001,2007-05-01,12.7
1,1001,2007-06-01,12.2
2,1001,2007-07-01,12.5
3,1001,2007-08-01,12.3
4,1001,2007-09-01,11.7
...,...,...,...
67764,16077,2021-01-01,7.3
67765,16077,2021-02-01,7.3
67766,16077,2021-03-01,7.2
67767,16077,2021-04-01,6.9


In [22]:
df_final_wide['date'] = pd.to_datetime(df_final_wide['date'])

In [23]:
# drop alq from original data and update directly from new data
df_final_wide = df_final_wide.drop(columns=['unemployment_rate'])

In [40]:
df_final_wide = pd.merge(df_final_wide, df_long, left_on=['ags5', 'date'], right_on=['ags5', 'date'])

In [41]:
df_final_wide

,ags2_x,ags5,date,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,...,realized_short_time_work_companies,realized_short_time_work_people,registerd_jobs,underemployment_without_short_time _work,unemployed,unemployment_benefit_entitled,unemployment_benefit_recipients,ags2_y,variable,value
0,1,1001,2007-05-01,4.0,2.0,21.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,unemployment_rate,12.7
1,1,1001,2007-06-01,8.0,6.0,18.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,unemployment_rate,12.2
2,1,1001,2007-07-01,9.0,0.0,18.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,unemployment_rate,12.5
3,1,1001,2007-08-01,6.0,4.0,10.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,unemployment_rate,12.3
4,1,1001,2007-09-01,11.0,4.0,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,unemployment_rate,11.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67363,16,16077,2020-12-01,2.0,1.0,6.0,115.0,99.0,158.0,44.0,...,NaN,NaN,702.0,4147.0,3015.0,6196.000000,979.0,16,unemployment_rate,6.7
67364,16,16077,2021-01-01,0.0,3.0,2.0,115.0,99.0,158.0,44.0,...,NaN,NaN,681.0,NaN,3287.0,6236.000000,NaN,16,unemployment_rate,7.3
67365,16,16077,2021-02-01,0.0,0.0,5.0,115.0,99.0,158.0,43.0,...,NaN,NaN,680.0,NaN,3270.0,6248.782315,NaN,16,unemployment_rate,7.3
67366,16,16077,2021-03-01,4.0,0.0,4.0,114.0,100.0,158.0,42.0,...,NaN,NaN,749.0,NaN,3222.0,6219.427429,NaN,16,unemployment_rate,7.2


### long

In [25]:
df_final_long.shape

(543439, 5)

In [26]:
df_long.shape

(67769, 3)

In [27]:
df_final_long['date'] = pd.to_datetime(df_final_long['date'])

In [28]:
# drop alq from original data
df_final_long = df_final_long[df_final_long['variable']!='unemployment_rate']

In [29]:
# # remove '-' and add 'd' for wide_to_long conversions
# df_wide = df_wide.rename(columns=lambda x: re.sub('-', '', x))
# df_wide = df_wide.rename(columns=lambda x: re.sub(r'([^\s]+)', 'd\\1', x))

In [30]:
# df_wide.reset_index(inplace=True)
# df_wide.rename(columns={"index": "ags5"}, inplace=True)

In [31]:
#df_wide

In [32]:
# df_wide = pd.wide_to_long(df_wide, i=['ags5'], stubnames='d', j='date').reset_index()

In [33]:
# df_wide.rename(columns={"d": "ags5"}, inplace=True)

In [34]:
#df_final_long

In [35]:
df_long['ags2'] = df_long['ags5'].astype(str).str[:-3]
df_long.rename(columns={"unemployment_rate": "value"}, inplace=True)
df_long['variable'] = 'unemployment_rate'

In [36]:
df_long = df_long[['ags2', 'ags5', 'variable', 'date', 'value']]

In [37]:
df_final_long = pd.concat([df_final_long, df_long], ignore_index=True)

In [38]:
df_final_long

,ags2,ags5,variable,date,value
0,1,1001,number_of_company_deletions,2007-01-01,4.0
1,1,1001,number_of_company_deletions,2007-02-01,5.0
2,1,1001,number_of_company_deletions,2007-03-01,7.0
3,1,1001,number_of_company_deletions,2007-04-01,5.0
4,1,1001,number_of_company_deletions,2007-05-01,4.0
...,...,...,...,...,...
599975,16,16077,unemployment_rate,2021-01-01,7.3
599976,16,16077,unemployment_rate,2021-02-01,7.3
599977,16,16077,unemployment_rate,2021-03-01,7.2
599978,16,16077,unemployment_rate,2021-04-01,6.9


In [42]:
# df_final_wide.to_csv('../../final_dfs/for_modeling/df_final_date_wide_2007.csv')
# df_final_long.to_csv('../../final_dfs/for_modeling/df_final_date_long_2007.csv')